# Initial data cleaning

In this notebook we will clean and prepare the data for the analysis. We will perform the following steps:

1. Load the data
2. Eliminate irrelevant columns
3. Check for missing values
4. Handle missing values
5. Check unique values per column and filter categorical data
6. Save the clean data

In [41]:
# Importing libraries

import pandas as pd

In [42]:
# Load data and eliminate irrelevant columns

df_train = pd.read_csv("../Data/Raw/train.csv")
df_test = pd.read_csv("../Data/Raw/test.csv")
df_train = df_train.drop(columns="id")
df_test = df_test.drop(columns="id")

# Duplicate removal

df_train = df_train.drop_duplicates()
df_test = df_test.drop_duplicates()

In [43]:
# Checking missing values per column

missing = df_train.isna().mean() * 100
missing

class                    0.000000
cap-diameter             0.000128
cap-shape                0.001283
cap-surface             21.528227
cap-color                0.000385
does-bruise-or-bleed     0.000257
gill-attachment         16.809280
gill-spacing            40.373988
gill-color               0.001829
stem-height              0.000000
stem-width               0.000000
stem-root               88.452732
stem-surface            63.551362
stem-color               0.001219
veil-type               94.884350
veil-color              87.936970
has-ring                 0.000770
ring-type                4.134818
spore-print-color       91.425482
habitat                  0.001444
season                   0.000000
dtype: float64

### Missing values conclussion:

As seen in the missing values analysis, there are columns missing a lot of values. We will drop any column with more than 30% missing values. We will then drop all the rows with missing values as there is a lot of data and we can afford to lose some of it.

In [44]:
high_missing = df_train.columns[missing > 30]

df_train = df_train.drop(columns=high_missing)
df_test = df_test.drop(columns=high_missing)

df_train = df_train.dropna()
df_test = df_test.dropna()

In [45]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1930108 entries, 0 to 3116944
Data columns (total 15 columns):
 #   Column                Dtype  
---  ------                -----  
 0   class                 object 
 1   cap-diameter          float64
 2   cap-shape             object 
 3   cap-surface           object 
 4   cap-color             object 
 5   does-bruise-or-bleed  object 
 6   gill-attachment       object 
 7   gill-color            object 
 8   stem-height           float64
 9   stem-width            float64
 10  stem-color            object 
 11  has-ring              object 
 12  ring-type             object 
 13  habitat               object 
 14  season                object 
dtypes: float64(3), object(12)
memory usage: 235.6+ MB


### Checking unique values per column

In [46]:
categorical = df_train.select_dtypes(include="object").columns.drop("class")

for column in categorical:
    unique = df_train[column].nunique()
    print(f"{column}: {unique}")

cap-shape: 48
cap-surface: 76
cap-color: 52
does-bruise-or-bleed: 21
gill-attachment: 64
gill-color: 49
stem-color: 50
has-ring: 18
ring-type: 37
habitat: 43
season: 4


In [47]:
for column in categorical:
    print(f"\nRows per category in: \n{df_train[column].value_counts().head(10)}")


Rows per category in: 
cap-shape
x    983649
f    450586
s    239475
b    127770
p     56234
c     55885
o     16166
d        48
e        34
t        28
Name: count, dtype: int64

Rows per category in: 
cap-surface
t    406544
s    343735
h    217348
g    203429
d    197858
y    185567
e     87558
w     85703
k     85415
i     71639
Name: count, dtype: int64

Rows per category in: 
cap-color
n    866525
w    240022
y    207793
e    137490
o    134369
g     80768
p     78930
k     45505
r     41267
u     40346
Name: count, dtype: int64

Rows per category in: 
does-bruise-or-bleed
f    1529827
t     400216
w          9
h          9
c          7
a          6
y          4
d          4
b          4
k          3
Name: count, dtype: int64

Rows per category in: 
gill-attachment
a    512171
d    418865
x    258594
s    241767
e    238960
p    226445
f     32915
c        55
u        42
t        32
Name: count, dtype: int64

Rows per category in: 
gill-color
w    586808
n    364863
y    336556


### Unique values conclussion:

Here it is clear that there are columns with a lot of unique values and most of that unique values only appear a couple of times. To reduce the amount of unique values we will unify all categories with less than 50000 appearances into a new category called 'Other'.

In [48]:
def unify_categories(df, column, threshold=50000):
    counts = df[column].value_counts()
    infrequent = counts[counts <= threshold].index
    df[column] = df[column].apply(lambda x: "Other" if x in infrequent else x)
    return df

for column in categorical:
    df_train = unify_categories(df_train, column)
    df_test = unify_categories(df_test, column)

In [49]:
# After unifying categories

for column in categorical:
    unique = df_train[column].nunique()
    print(f"{column}: {unique}")

cap-shape: 7
cap-surface: 11
cap-color: 8
does-bruise-or-bleed: 3
gill-attachment: 7
gill-color: 7
stem-color: 7
has-ring: 3
ring-type: 7
habitat: 6
season: 4


In [50]:
for column in categorical:
    print(f"\nRows per category in: \n{df_train[column].value_counts().head(10)}")


Rows per category in: 
cap-shape
x        983649
f        450586
s        239475
b        127770
p         56234
c         55885
Other     16509
Name: count, dtype: int64

Rows per category in: 
cap-surface
t    406544
s    343735
h    217348
g    203429
d    197858
y    185567
e     87558
w     85703
k     85415
i     71639
Name: count, dtype: int64

Rows per category in: 
cap-color
n        866525
w        240022
y        207793
Other    184211
e        137490
o        134369
g         80768
p         78930
Name: count, dtype: int64

Rows per category in: 
does-bruise-or-bleed
f        1529827
t         400216
Other         65
Name: count, dtype: int64

Rows per category in: 
gill-attachment
a        512171
d        418865
x        258594
s        241767
e        238960
p        226445
Other     33306
Name: count, dtype: int64

Rows per category in: 
gill-color
w        586808
n        364863
y        336556
p        281212
Other    154458
o        103729
g        102482
Name: count

In [51]:
# Duplicate removal

df_train = df_train.drop_duplicates()
df_test = df_test.drop_duplicates()

In [52]:
df_train.to_csv("../Data/PartiallyClean/train.csv", index=False)
df_test.to_csv("../Data/PartiallyClean/test.csv", index=False)